# Alzheimer's Disease Prediction - Ensemble Learning Approaches

This notebook explores various ensemble learning techniques to potentially improve the model performance for Alzheimer's disease prediction, with a focus on voting classifiers. We'll implement and compare several approaches including:

1. Hard voting classifiers
2. Soft voting classifiers
3. Weighted voting classifiers
4. Stacking ensemble models
5. Combining with dimensionality reduction techniques

The goal is to leverage the strengths of multiple models to achieve better predictive performance for Alzheimer's disease diagnosis.

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os

from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report, roc_curve, precision_recall_curve, auc
from sklearn.preprocessing import StandardScaler

# Import ensemble models
from sklearn.ensemble import VotingClassifier, StackingClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import xgboost as xgb
from catboost import CatBoostClassifier

# Import dimensionality reduction methods for combined analysis
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline

# Set plot styling
plt.style.use('seaborn-v0_8-whitegrid')
sns.set(font_scale=1.2)

# For displaying plots inline
%matplotlib inline

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

## 1. Loading and Preparing the Data

We'll load the processed data from the previous feature engineering approaches and prepare it for our ensemble models.

In [3]:
# Load the feature-engineered data
# First check which approach performed better in previous notebooks
fe_statistical = pd.read_csv('fe_statistical_approach.csv')
fe_model_based = pd.read_csv('fe_model_based_approach.csv')

# We'll use both approaches and compare results
# Approach 1: Statistical feature engineering
data_statistical = fe_statistical.copy()
X_statistical = data_statistical.drop("Alzheimer's Diagnosis", axis=1)
y = data_statistical["Alzheimer's Diagnosis"]

# Approach 2: Model-based feature engineering
data_model_based = fe_model_based.copy()
X_model_based = data_model_based.drop("Alzheimer's Diagnosis", axis=1)

# Split the data into training and testing sets
X_stat_train, X_stat_test, y_train, y_test = train_test_split(X_statistical, y, test_size=0.2, random_state=42, stratify=y)
X_model_train, X_model_test, y_train_dup, y_test_dup = train_test_split(X_model_based, y, test_size=0.2, random_state=42, stratify=y)

# Feature scaling
scaler_stat = StandardScaler()
X_stat_train_scaled = scaler_stat.fit_transform(X_stat_train)
X_stat_test_scaled = scaler_stat.transform(X_stat_test)

scaler_model = StandardScaler()
X_model_train_scaled = scaler_model.fit_transform(X_model_train)
X_model_test_scaled = scaler_model.transform(X_model_test)

print(f"Statistical approach features shape: {X_stat_train_scaled.shape}")
print(f"Model-based approach features shape: {X_model_train_scaled.shape}")

Statistical approach features shape: (59426, 23)
Model-based approach features shape: (59426, 19)


## 2. Load Pre-trained Models from Previous Analysis

We'll load the best performing models from our previous analyses to include them in our ensemble models.

In [4]:
# Function to safely load models
def load_model(file_path):
    try:
        with open(file_path, 'rb') as f:
            return pickle.load(f)
    except FileNotFoundError:
        print(f"Warning: Model file {file_path} not found. Creating new instance.")
        return None

# Load pre-trained models if available
lr_model = load_model('logistic_regression_best_model.pkl')
rf_model = load_model('random_forest_best_model.pkl')
gb_model = load_model('gradient_boosting_best_model.pkl')

# Create base model instances if pre-trained models not available
if lr_model is None:
    lr_model = LogisticRegression(random_state=42, max_iter=1000)
if rf_model is None:
    rf_model = RandomForestClassifier(random_state=42)
if gb_model is None:
    gb_model = GradientBoostingClassifier(random_state=42)

# Create additional models for ensembling
svm_model = SVC(probability=True, random_state=42)
xgb_model = xgb.XGBClassifier(random_state=42)
cat_model = CatBoostClassifier(random_state=42, verbose=0)

# List all models for easy reference
models = {
    'Logistic Regression': lr_model,
    'Random Forest': rf_model,
    'Gradient Boosting': gb_model,
    'SVM': svm_model,
    'XGBoost': xgb_model,
    'CatBoost': cat_model
}

## 3. Define Evaluation Framework

We'll create a comprehensive evaluation framework for our ensemble models, including cross-validation and performance metrics.

In [7]:
# Function to evaluate models with cross-validation
def evaluate_model_cv(model, X, y, cv=5):
    cv_results = cross_validate(
        model, X, y, 
        cv=StratifiedKFold(n_splits=cv, shuffle=True, random_state=42),
        scoring=['accuracy', 'precision', 'recall', 'f1', 'roc_auc'],
        return_train_score=True
    )
    
    # Compute mean and std of metrics
    metrics = {}
    for metric in ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']:
        metrics[f'test_{metric}'] = {
            'mean': cv_results[f'test_{metric}'].mean(),
            'std': cv_results[f'test_{metric}'].std()
        }
    
    return metrics

# Function for detailed model evaluation on test set
# Function for detailed model evaluation on test set
def evaluate_model_test(model, X_train, X_test, y_train, y_test):
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Check if the model supports probability predictions
    if hasattr(model, 'predict_proba'):
        y_prob = model.predict_proba(X_test)[:, 1]
        roc_auc = roc_auc_score(y_test, y_prob)
    elif hasattr(model, 'decision_function'):  # For SVM without probability
        y_prob = model.decision_function(X_test)
        roc_auc = roc_auc_score(y_test, y_prob)
    else:
        # For hard voting and other models without probability outputs
        y_prob = y_pred  # Just use the predictions (this won't be a proper probability)
        print(f"Warning: {type(model).__name__} doesn't support probability predictions. ROC AUC may not be valid.")
        roc_auc = None
    
    # Calculate metrics
    metrics = {
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred),
        'recall': recall_score(y_test, y_pred),
        'f1': f1_score(y_test, y_pred),
        'roc_auc': roc_auc
    }
    
    # Generate confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    
    return metrics, cm, y_prob

# Function to plot confusion matrix
def plot_confusion_matrix(cm, model_name):
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.title(f'Confusion Matrix - {model_name}')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

# Function to plot ROC curve
def plot_roc_curve(y_test, y_prob_dict, model_names):
    plt.figure(figsize=(10, 8))
    for model_name in model_names:
        y_prob = y_prob_dict[model_name]
        fpr, tpr, _ = roc_curve(y_test, y_prob)
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, lw=2, label=f'{model_name} (AUC = {roc_auc:.3f})')
    
    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curves')
    plt.legend(loc='lower right')
    plt.grid(True)
    plt.show()

## 4. Hard Voting Ensemble

In hard voting, the predicted class label for each test instance is the one that receives the highest number of votes from the base classifiers.

In [8]:
# Create hard voting classifier
hard_voting_clf = VotingClassifier(
    estimators=[
        ('lr', lr_model),
        ('rf', rf_model),
        ('gb', gb_model),
        ('svm', svm_model),
        ('xgb', xgb_model),
    ],
    voting='hard'
)

# Evaluate with cross-validation
print("Evaluating Hard Voting Classifier with statistical features:")
hard_voting_cv_stat = evaluate_model_cv(hard_voting_clf, X_stat_train_scaled, y_train)
for metric, values in hard_voting_cv_stat.items():
    print(f"{metric}: {values['mean']:.4f} (±{values['std']:.4f})")

print("\nEvaluating Hard Voting Classifier with model-based features:")
hard_voting_cv_model = evaluate_model_cv(hard_voting_clf, X_model_train_scaled, y_train)
for metric, values in hard_voting_cv_model.items():
    print(f"{metric}: {values['mean']:.4f} (±{values['std']:.4f})")

# Evaluate on test set
print("\nTest performance with statistical features:")
hard_metrics_stat, hard_cm_stat, hard_prob_stat = evaluate_model_test(
    hard_voting_clf, X_stat_train_scaled, X_stat_test_scaled, y_train, y_test
)
for metric, value in hard_metrics_stat.items():
    print(f"{metric}: {value:.4f}")

plot_confusion_matrix(hard_cm_stat, "Hard Voting Ensemble")

Evaluating Hard Voting Classifier with statistical features:


AttributeError: VotingClassifier has none of the following attributes: decision_function, predict_proba.

## 5. Soft Voting Ensemble

In soft voting, each classifier provides a probability estimate for each class, and these probabilities are averaged to find the final class prediction.

In [ ]:
# Create soft voting classifier
# Note: All classifiers need to support predict_proba
soft_voting_clf = VotingClassifier(
    estimators=[
        ('lr', lr_model),
        ('rf', rf_model),
        ('gb', gb_model),
        ('svm', svm_model),
        ('xgb', xgb_model),
    ],
    voting='soft'
)

# Evaluate with cross-validation
print("Evaluating Soft Voting Classifier with statistical features:")
soft_voting_cv_stat = evaluate_model_cv(soft_voting_clf, X_stat_train_scaled, y_train)
for metric, values in soft_voting_cv_stat.items():
    print(f"{metric}: {values['mean']:.4f} (±{values['std']:.4f})")

print("\nEvaluating Soft Voting Classifier with model-based features:")
soft_voting_cv_model = evaluate_model_cv(soft_voting_clf, X_model_train_scaled, y_train)
for metric, values in soft_voting_cv_model.items():
    print(f"{metric}: {values['mean']:.4f} (±{values['std']:.4f})")

# Evaluate on test set
print("\nTest performance with statistical features:")
soft_metrics_stat, soft_cm_stat, soft_prob_stat = evaluate_model_test(
    soft_voting_clf, X_stat_train_scaled, X_stat_test_scaled, y_train, y_test
)
for metric, value in soft_metrics_stat.items():
    print(f"{metric}: {value:.4f}")

plot_confusion_matrix(soft_cm_stat, "Soft Voting Ensemble")

Evaluating Soft Voting Classifier with statistical features:


## 6. Weighted Voting Ensemble

In weighted voting, we assign different weights to each classifier based on their performance, giving more influence to better-performing models.

In [ ]:
# Train individual models to determine weights
model_metrics = {}
model_probs = {}

for name, model in models.items():
    # Skip CatBoost for weighted voting (to be used later in stacking)
    if name == 'CatBoost':
        continue
        
    print(f"Training {name}...")
    metrics, _, probs = evaluate_model_test(
        model, X_stat_train_scaled, X_stat_test_scaled, y_train, y_test
    )
    model_metrics[name] = metrics
    model_probs[name] = probs

# Display performance metrics for each model
metrics_df = pd.DataFrame({
    name: {
        metric: value for metric, value in metrics.items()
    } for name, metrics in model_metrics.items()
})

print("\nIndividual model performance:")
print(metrics_df.T)

# Assign weights based on AUC scores
weights = {name: metrics['roc_auc'] for name, metrics in model_metrics.items()}

# Create weighted voting classifier
weighted_voting_clf = VotingClassifier(
    estimators=[
        ('lr', lr_model),
        ('rf', rf_model),
        ('gb', gb_model),
        ('svm', svm_model),
        ('xgb', xgb_model),
    ],
    voting='soft',
    weights=[weights['Logistic Regression'], 
             weights['Random Forest'], 
             weights['Gradient Boosting'], 
             weights['SVM'], 
             weights['XGBoost']]
)

# Evaluate weighted voting
print("\nEvaluating Weighted Voting Classifier:")
weighted_metrics, weighted_cm, weighted_prob = evaluate_model_test(
    weighted_voting_clf, X_stat_train_scaled, X_stat_test_scaled, y_train, y_test
)
for metric, value in weighted_metrics.items():
    print(f"{metric}: {value:.4f}")

plot_confusion_matrix(weighted_cm, "Weighted Voting Ensemble")

# Add probabilities for the weighted voting model
model_probs['Weighted Voting'] = weighted_prob

# Plot ROC curves
plot_roc_curve(y_test, model_probs, list(model_probs.keys()))

## 7. Stacking Ensemble Model

Stacking involves training a meta-model that combines the predictions of multiple base models, potentially capturing more complex patterns than simple voting.

In [ ]:
# Create a stacking classifier
# We'll use logistic regression as the final estimator
estimators = [
    ('lr', lr_model),
    ('rf', rf_model),
    ('gb', gb_model),
    ('svm', svm_model),
    ('xgb', xgb_model),
]

stack_clf = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(random_state=42),
    cv=5,
    stack_method='predict_proba'
)

# Evaluate stacking classifier
print("Evaluating Stacking Classifier with statistical features:")
stack_metrics_stat, stack_cm_stat, stack_prob_stat = evaluate_model_test(
    stack_clf, X_stat_train_scaled, X_stat_test_scaled, y_train, y_test
)
for metric, value in stack_metrics_stat.items():
    print(f"{metric}: {value:.4f}")

plot_confusion_matrix(stack_cm_stat, "Stacking Ensemble")

print("\nEvaluating Stacking Classifier with model-based features:")
stack_metrics_model, stack_cm_model, stack_prob_model = evaluate_model_test(
    stack_clf, X_model_train_scaled, X_model_test_scaled, y_train, y_test
)
for metric, value in stack_metrics_model.items():
    print(f"{metric}: {value:.4f}")

# Add stacking model probabilities
model_probs['Stacking'] = stack_prob_stat

# Plot updated ROC curves with stacking model included
plot_roc_curve(y_test, model_probs, list(model_probs.keys()))

## 8. Combining Dimensionality Reduction with Ensemble Learning

Here we'll explore whether combining dimensionality reduction techniques with our best performing ensemble model can further improve performance.

In [ ]:
# Create pipelines combining dimensionality reduction with ensemble methods
# We'll use the best performing ensemble method from above

# Determine best ensemble model (let's assume stacking for this example)
best_ensemble_model = stack_clf

# Create pipelines with different dimensionality reduction techniques
pipelines = {
    'PCA + Ensemble': Pipeline([
        ('pca', PCA(n_components=0.95)),  # Keep 95% of variance
        ('ensemble', best_ensemble_model)
    ]),
    'LDA + Ensemble': Pipeline([
        ('lda', LDA(n_components=1)),  # Binary classification -> 1 component
        ('ensemble', best_ensemble_model)
    ]),
    'SelectKBest + Ensemble': Pipeline([
        ('select', SelectKBest(f_classif, k=10)),  # Select top 10 features
        ('ensemble', best_ensemble_model)
    ]),
    'Ensemble Only': best_ensemble_model
}

# Evaluate all pipelines
pipeline_metrics = {}
pipeline_probs = {}

for name, pipeline in pipelines.items():
    print(f"\nEvaluating {name}:")
    metrics, cm, probs = evaluate_model_test(
        pipeline, X_stat_train_scaled, X_stat_test_scaled, y_train, y_test
    )
    pipeline_metrics[name] = metrics
    pipeline_probs[name] = probs
    
    for metric, value in metrics.items():
        print(f"{metric}: {value:.4f}")
    
    plot_confusion_matrix(cm, name)

# Compare pipeline performance
pipeline_df = pd.DataFrame({
    name: {
        metric: value for metric, value in metrics.items()
    } for name, metrics in pipeline_metrics.items()
})

print("\nPipeline performance comparison:")
print(pipeline_df.T)

# Plot ROC curves for the pipelines
plot_roc_curve(y_test, pipeline_probs, list(pipeline_probs.keys()))

## 9. Comprehensive Model Comparison

Let's compare all the models we've developed, including individual models and ensembles, to identify the best approach.

In [ ]:
# Combine all metrics into a single dataframe
all_metrics = {}

# Add individual model metrics
all_metrics.update(model_metrics)

# Add ensemble model metrics
all_metrics['Hard Voting'] = hard_metrics_stat
all_metrics['Soft Voting'] = soft_metrics_stat
all_metrics['Weighted Voting'] = weighted_metrics
all_metrics['Stacking'] = stack_metrics_stat

# Add pipeline metrics
all_metrics.update(pipeline_metrics)

# Create dataframe and sort by F1 score
all_metrics_df = pd.DataFrame({
    name: {
        metric: value for metric, value in metrics.items()
    } for name, metrics in all_metrics.items()
})

print("\nAll model performance metrics:")
print(all_metrics_df.T.sort_values('f1', ascending=False))

# Visualize the comparison with a heatmap
plt.figure(figsize=(14, 10))
sns.heatmap(all_metrics_df.T, annot=True, cmap='viridis', fmt='.3f', linewidths=0.5)
plt.title('Performance Comparison of All Models')
plt.tight_layout()
plt.show()

# Bar plot comparing F1 scores
plt.figure(figsize=(12, 8))
models_sorted = all_metrics_df.T.sort_values('f1', ascending=False).index
sns.barplot(x=all_metrics_df.loc['f1', models_sorted], y=models_sorted)
plt.title('Model Comparison by F1 Score')
plt.xlabel('F1 Score')
plt.xlim(min(all_metrics_df.loc['f1']) - 0.05, 1.0)
plt.grid(True, axis='x')
plt.tight_layout()
plt.show()

## 10. Save the Best Performing Model

We'll save the best performing model for future predictions.

In [ ]:
# Identify the best model based on F1 score
best_model_name = all_metrics_df.T.sort_values('f1', ascending=False).index[0]
print(f"The best performing model is: {best_model_name}")

# Get the best model
if best_model_name in pipelines:
    best_model = pipelines[best_model_name]
    # Ensure the model is fitted
    best_model.fit(X_stat_train_scaled, y_train)
elif best_model_name == 'Hard Voting':
    best_model = hard_voting_clf
    best_model.fit(X_stat_train_scaled, y_train)
elif best_model_name == 'Soft Voting':
    best_model = soft_voting_clf
    best_model.fit(X_stat_train_scaled, y_train)
elif best_model_name == 'Weighted Voting':
    best_model = weighted_voting_clf
    best_model.fit(X_stat_train_scaled, y_train)
elif best_model_name == 'Stacking':
    best_model = stack_clf
    best_model.fit(X_stat_train_scaled, y_train)
else:
    best_model = models[best_model_name]
    best_model.fit(X_stat_train_scaled, y_train)

# Save the best model
best_model_path = 'best_ensemble_model.pkl'
with open(best_model_path, 'wb') as f:
    pickle.dump(best_model, f)

print(f"Best model saved to {best_model_path}")

## 11. Conclusion and Next Steps

In this notebook, we explored various ensemble learning approaches for Alzheimer's disease prediction, including voting classifiers and stacking ensembles. We also combined these approaches with dimensionality reduction techniques to potentially improve performance further.

Key findings:
1. The best performing model was [will be determined by execution]
2. Ensemble models generally outperformed individual models
3. Combining dimensionality reduction with ensemble learning [will be determined by execution]

Next steps could include:
1. Further fine-tuning of the best performing model
2. Exploring more complex ensemble architectures
3. Developing a deployment strategy for clinical use
4. Conducting external validation on independent datasets